# Download and convert to Geopackage

This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Geopackage Database for future use.
Geopackage is a convenient data storage format, based on SQL, and is compatible with QGIS.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [2]:
# reading the area of interest polygon (could have been set)
# gdf_geom = gpd.read_file('/home/hysope2/STUDIES/SWOT_Panama/DATA/aoi.gpkg')
gdf_geom = gpd.read_file("../data/aoi.gpkg")

# Limiting time period
dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)

## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=1,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

Downloaded products:   0%|                                                                                    …

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a geopackage database.
This geopackage format is quite efficient (though not the most efficient), and may easily be visualized in, e.g., QGIS
We are using the same geodataframe to limit the data to the area of interest

In [4]:
from pixcdust.converters.gpkg import Nc2GpkgConverter
from glob import glob

In [5]:
pixc = Nc2GpkgConverter(
            glob(pixcdownloader.path_download+'/*/*nc'),
            "/tmp/pixc_gpkg.gpkg",
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:55<00:00, 57.95s/it]


database has been succesfully created, we can remove the raw files

In [6]:
# import shutil
# shutil.rmtree('/tmp/pixc')

# Read the database
Previous steps are not necessary

Now we can open this database in a GeoDataFrame, load it in, e.g., QGIS, etc.

In [7]:
from pixcdust.readers.gpkg import GpkgReader

# nb: you may specify 
pixc_read = GpkgReader(
    "/tmp/pixc_gpkg.gpkg"
)
pixc_read.read()
pixc_read.data

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.12it/s]


<xarray.Dataset> Size: 109MB
Dimensions:         (index: 1519109)
Coordinates:
  * index           (index) int64 12MB 0 1 2 3 ... 1519106 1519107 1519108
Data variables:
    height          (index) float64 12MB 207.3 207.7 208.4 ... 276.1 265.7 260.5
    sig0            (index) float64 12MB -0.002999 0.2483 ... 1.209 0.4759
    classification  (index) float64 12MB 1.0 1.0 2.0 3.0 2.0 ... 1.0 1.0 1.0 1.0
    geoid           (index) float64 12MB 49.46 49.46 49.46 ... 49.4 49.4 49.4
    latitude        (index) float64 12MB 43.52 43.52 43.52 ... 43.69 43.69 43.69
    longitude       (index) float64 12MB 1.384 1.384 1.385 ... 1.51 1.51 1.51
    wse             (index) float64 12MB 157.9 158.3 159.0 ... 226.8 216.3 211.1
    geometry        (index) geometry 12MB <class 'xarray.core.extension_array...

Enjoy!